In [95]:
import requests  #向服务器发送请求的模块（相当于浏览)
from bs4 import BeautifulSoup  #用于解析和提取数据
import openpyxl  #读写excel文件
import time

lst=[]
def send_requests(id,page):
    url='https://www.jobui.com/jobs?jobKw=%E8%80%81%E5%B8%88&cityKw=%E6%B7%B1%E5%9C%B3&degreeType=%E5%88%9D%E4%B8%AD'.format(id,page) #相当于浏览器的地址栏
    #用于反爬的一种手段―(伪装成浏览器)
    headers= {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}
    resp=requests.get(url,headers=headers) #相当于在浏览器的地址栏中输入网址之后回车的过程
    return resp.text

def parse_html(html): #html代表是从服务器端响应回来的数据(内容)
    bs=BeautifulSoup(html,'html.parser') #得到BeautifulSoup的对象
    job_lst=bs.find_all('div',class_='c-job-list') #得到包含职位的div，--》15
    for item in job_lst: #分别遍历每一个职位数据每个item就是一个c-job-lst
        name=item.find('h3').text #职位名称
        div_tag1=item.find('div',class_='job-desc')
        span_tag1=div_tag1.find_all('span')#标签
        url1=item.find('a',class_='job-name')['href'] #提取class样式为job-name的a标签，获取属性为href的值
        company_tag = item.find('a', class_='job-company-name')
        if company_tag is not None:
            company_name = company_tag.text
        address=company_name[:2]
        company_desc = item.find('span', class_='job-desc fwb')
        company_description=""
        if company_desc is not None:
            company_description = company_desc.text
        url1="https://www.jobui.com"+url1
        url2=item.find('a',class_='job-company-name')['href']
        url2="https://www.jobui.com"+url2
        lst.append([name,span_tag1[0].text,span_tag1[2].text,span_tag1[4].text,company_name,address,company_description,url1,url2])

#存储
def save(lst):
    filename = 'C:\\Users\\lenovo\\Desktop\\2.xlsx'
    # 打开现有的Excel文件
    try:
        wb = openpyxl.load_workbook(filename)
        sheet = wb.active
    except FileNotFoundError:
        wb = openpyxl.Workbook()
        sheet = wb.active
    
    # 从新的行开始追加数据
    row = sheet.max_row + 1
    for item in lst:
        if isinstance(item, list):
            for i, subitem in enumerate(item):
                sheet.cell(row=row, column=i+1, value=subitem)
        else:
            sheet.cell(row=row, column=1, value=item)
        row += 1
    
    # 保存文件
    wb.save(filename)

#启动爬虫程序
def start(id,pages):

    for page in range(1,pages+1) :
        resp_data=send_requests(id,page) #发送请求
        parse_html(resp_data)
        time.sleep(0.5)
    save(lst)


if __name__ == '__main__':
        pages=5
        #  深圳北京杭州山东 重庆东莞
        start(id,pages) 